In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.layers import Conv1D, Dense, Dropout, Flatten, BatchNormalization, MaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
import scipy.io as scipy
from scipy import signal
from sklearn.model_selection import train_test_split


In [ ]:
data_paths = ['Training_2/', 'WFDB/']
atrial_fibrillation = '164889003'
atrial_fib_and_flutter = '195080001'


df = pd.DataFrame()
classes = []

X_all = np.zeros((13797, 500, 12))
index = 0

for datapath in data_paths:
    files = os.listdir(datapath)
    for file in files:
        if file.endswith('.mat'):
            mat = scipy.loadmat(datapath+file)
            
            arr = list(mat.values())
            arr = arr[0]
            resampled = signal.resample(arr[0][0:1500], 500)
            # plt.plot(resampled)
            # plt.show()

            # arr = pad_sequences(arr, maxlen = 1000, truncating = 'post', padding = 'post')
            swapped = np.swapaxes([resampled], 0 , 1)
            X_all[index] = swapped
            index += 1
            
            df_sub = pd.DataFrame({'I':[arr[0]], 'II':[arr[1]], 'III':[arr[2]], 'aVR':[arr[3]],
                                  'aVL':[arr[4]], 'aVF':[arr[5]], 'V1':[arr[6]], 'V2':[arr[7]], 'V3':[arr[8]],
                                  'V4':[arr[9]], 'V5':[arr[10]], 'V6':[arr[11]]})
            df = pd.concat([df, df_sub],axis = 0)
            
            
        if file.endswith('.hea'):
            input_header_file = datapath+file
            with open(input_header_file,'r') as f:
                header_data=f.readlines()
                dx = header_data[15][5:].strip('\n').split(',')
                dx = int(dx[0])
                if dx == int(atrial_fibrillation) or dx == int(atrial_fib_and_flutter):
                    classes.append(1)
                else:
                    classes.append(0)
                    
df.index = range(0,len(df))
df['Output'] = classes
classes = np.array(classes)

X_train, X_test,y_train,y_test=train_test_split(X_all, classes)

# Simple CNN for ECG data
model = Sequential()
model.add(Conv1D(filters = 128, kernel_size = 5, activation = 'relu', input_shape = (500,12)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))
# compile the model - use categorical crossentropy, and the adam optimizer
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.fit(X_train, y_train, epochs = 20, batch_size = 32, verbose = 1)
yhat = model.predict(X_test, verbose = 0)


Epoch 1/20
324/324 [==============================] - 34s 4ms/step - loss: 14.8680 - accuracy: 0.9527
Epoch 2/20
324/324 [==============================] - 1s 4ms/step - loss: 6.3658 - accuracy: 0.9539
Epoch 3/20
324/324 [==============================] - 1s 4ms/step - loss: 13.5208 - accuracy: 0.9561
Epoch 4/20
324/324 [==============================] - 1s 4ms/step - loss: 21.5644 - accuracy: 0.9565
Epoch 5/20
324/324 [==============================] - 1s 4ms/step - loss: 10.8394 - accuracy: 0.9588
Epoch 6/20
324/324 [==============================] - 1s 4ms/step - loss: 13.1409 - accuracy: 0.9556
Epoch 7/20
324/324 [==============================] - 1s 4ms/step - loss: 16.7360 - accuracy: 0.9556
Epoch 8/20
324/324 [==============================] - 1s 4ms/step - loss: 13.9120 - accuracy: 0.9590
Epoch 9/20
324/324 [==============================] - 1s 4ms/step - loss: 16.2540 - accuracy: 0.9514
Epoch 10/20
324/324 [==============================] - 1s 4ms/step - loss: 36.8991 - accura

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
print(score)


total_correct = 0
sum = 0
for i in range(len(yhat)):
    if yhat[i] == y_test[i] and yhat[i] == 1:
        sum += 1
        total_correct += 1
    elif y_test[i] == 1:
      sum += 1

print(total_correct, sum)

accuracy: 96.93%
[32.978275299072266, 0.9692753553390503]
2 85


In [10]:
from keras.models import model_from_yaml
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
!unzip -u "/content/WFDB.zip" -d "/content"

Streaming output truncated to the last 5000 lines.
  inflating: /content/WFDB/E07845.hea  
  inflating: /content/WFDB/E07845.mat  
  inflating: /content/WFDB/E07846.hea  
  inflating: /content/WFDB/E07846.mat  
  inflating: /content/WFDB/E07847.hea  
  inflating: /content/WFDB/E07847.mat  
  inflating: /content/WFDB/E07848.hea  
  inflating: /content/WFDB/E07848.mat  
  inflating: /content/WFDB/E07849.hea  
  inflating: /content/WFDB/E07849.mat  
  inflating: /content/WFDB/E07850.hea  
  inflating: /content/WFDB/E07850.mat  
  inflating: /content/WFDB/E07851.hea  
  inflating: /content/WFDB/E07851.mat  
  inflating: /content/WFDB/E07852.hea  
  inflating: /content/WFDB/E07852.mat  
  inflating: /content/WFDB/E07853.hea  
  inflating: /content/WFDB/E07853.mat  
  inflating: /content/WFDB/E07854.hea  
  inflating: /content/WFDB/E07854.mat  
  inflating: /content/WFDB/E07855.hea  
  inflating: /content/WFDB/E07855.mat  
  inflating: /content/WFDB/E07856.hea  
  inflating: /content/WFDB/E0